

## Trainee : AbdalRhman Gameel Ahmed Hebishy (G5)



# Advanced NumPy

Welcome to the advanced notebook in NumPy! If you have comments or suggestions, please don’t hesitate to share it in the end of the session!

![alt_text](S1-pics\NumPy_logo.png)

In [99]:
import numpy as np

-----

# Section 1: Broadcasting
Broadcasting is simply a set of rules for applying binary ufuncs (e.g., addition, subtraction, multiplication, etc.) on arrays of different sizes.

## Rules of Broadcasting

Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays:

- Rule 1: If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is *padded* with ones on its leading (left) side.
- Rule 2: If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
- Rule 3: If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

---

### Broadcasting example 1

Let's look at adding a two-dimensional array to a one-dimensional array:

In [100]:
M = np.ones((2, 3))
M

array([[1., 1., 1.],
       [1., 1., 1.]])

In [101]:
a = np.arange(3)
a

array([0, 1, 2])

Let's consider an operation on these two arrays. The shape of the arrays are

- ``M.shape = (2, 3)``
- ``a.shape = (3,)``

We see by rule 1 that the array ``a`` has fewer dimensions, so we pad it on the left with ones:

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``

By rule 2, we now see that the first dimension disagrees, so we stretch this dimension to match:

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``

The shapes match, and we see that the final shape will be ``(2, 3)``:

In [102]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.]])

### Broadcasting example 2

Let's take a look at an example where both arrays need to be broadcast:

In [103]:
a = np.arange(3).reshape((3, 1))
a

array([[0],
       [1],
       [2]])

In [104]:
b = np.arange(3)
b

array([0, 1, 2])

Again, we'll start by writing out the shape of the arrays:

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``

Rule 1 says we must pad the shape of ``b`` with ones:

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``

And rule 2 tells us that we upgrade each of these ones to match the corresponding size of the other array:

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``

Because the result matches, these shapes are compatible. We can see this here:

In [105]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### Broadcasting example 3

Now let's take a look at an example in which the two arrays are not compatible:

In [106]:
M = np.ones((3, 2))
M


array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [107]:
a = np.arange(3)
a

array([0, 1, 2])

This is just a slightly different situation than in the first example: the matrix ``M`` is transposed.
How does this affect the calculation? The shape of the arrays are

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``

Again, rule 1 tells us that we must pad the shape of ``a`` with ones:

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``

By rule 2, the first dimension of ``a`` is stretched to match that of ``M``:

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``

Now we hit rule 3–the final shapes do not match, so these two arrays are incompatible, as we can observe by attempting this operation:

In [108]:
new_var = M + a[:3].reshape((3, 1))
new_var

array([[1., 1.],
       [2., 2.],
       [3., 3.]])

Note the potential confusion here:
you could imagine making ``a`` and ``M`` compatible by, say, padding ``a``'s shape with ones on the right rather than the left.
But this is not how the broadcasting rules work!
That sort of flexibility might be useful in some cases, but it would lead to potential areas of ambiguity.

If right-side padding is what you'd like, you can do this explicitly by reshaping the array (we'll use the ``np.newaxis`` ):

In [109]:
a[:, np.newaxis].shape

(3, 1)

In [110]:
M + a[:, np.newaxis]

array([[1., 1.],
       [2., 2.],
       [3., 3.]])

Also note that while we've been focusing on the ``+`` operator here, these broadcasting rules apply to *any* binary ``ufunc``.
For example, here is the ``logaddexp(a, b)`` function, which computes ``log(exp(a) + exp(b))`` with more precision than the naive approach:

In [111]:
np.logaddexp(M, a[:, np.newaxis])

array([[1.31326169, 1.31326169],
       [1.69314718, 1.69314718],
       [2.31326169, 2.31326169]])

---
## Broadcasting in Practice
### Centering an array

One commonly seen example is when centering an array of data.

Imagine you have an array of `10 observations`, each of which consists of `3 values`.

In [112]:
X = np.random.random((10, 3))
X

array([[0.75641462, 0.14302242, 0.63724424],
       [0.77853413, 0.7199452 , 0.39237667],
       [0.81277831, 0.78938951, 0.83422381],
       [0.1466074 , 0.81878254, 0.19476508],
       [0.91288997, 0.39850758, 0.40930401],
       [0.18199228, 0.15886372, 0.36117546],
       [0.58126465, 0.82904687, 0.5690743 ],
       [0.9868596 , 0.71971307, 0.6992618 ],
       [0.50835862, 0.76167599, 0.47109208],
       [0.26600723, 0.93489254, 0.22331695]])

We can compute the mean of each feature using the ``mean`` aggregate across the first dimension:

In [113]:
Xmean = X.mean(0)
Xmean

array([0.59317068, 0.62738394, 0.47918344])

And now we can center the ``X`` array by subtracting the mean (this is a broadcasting operation):

In [114]:
X_centered = X - Xmean
X_centered

array([[ 0.16324394, -0.48436152,  0.1580608 ],
       [ 0.18536345,  0.09256125, -0.08680677],
       [ 0.21960763,  0.16200557,  0.35504037],
       [-0.44656328,  0.19139859, -0.28441836],
       [ 0.31971929, -0.22887637, -0.06987943],
       [-0.4111784 , -0.46852022, -0.11800798],
       [-0.01190603,  0.20166292,  0.08989086],
       [ 0.39368892,  0.09232912,  0.22007836],
       [-0.08481206,  0.13429205, -0.00809136],
       [-0.32716345,  0.3075086 , -0.25586649]])

To double-check that we've done this correctly, we can check that the centered array has near zero mean:

In [115]:
X_centered.mean(0)

array([ 1.22124533e-16,  6.66133815e-17, -3.33066907e-17])

----

# Section 2: More useful array operations
This section covers `maximum`, `minimum`, `sum`, `mean`, `product`, `standard deviation` and more

NumPy also performs aggregation functions. In addition to `min`, `max`, and `sum`,
- you can easily run `mean` to get the average,
-`prod` to get the result of multiplying the elements together,
-`std` to get the standard deviation, and more.

Once you’ve created your arrays, you can start to work with them.

In [116]:
data = np.array([1.0, 2.0, 3.0])
data

array([1., 2., 3.])

In [117]:
data.max()

3.0

In [118]:
data.min()

1.0

In [119]:
data.sum()

6.0

In [120]:
data.mean()

2.0

In [121]:
data.std()

0.816496580927726

In [122]:
data.prod()

6.0

![alt_text](S1-pics\data.png)

Let’s start with this array, called `a`

It’s very common to want to aggregate along a row or column. By default, every NumPy aggregation function will return the aggregate of the entire array.

**Try It**
- To find the sum or the minimum of the elements in your array



In [123]:
a = np.array([[0.45053314, 0.17296777, 0.34376245, 0.5510652],
               [0.54627315, 0.05093587, 0.40067661, 0.55645993],
               [0.12697628, 0.82485143, 0.26590556, 0.56917101]])

In [124]:
# Code here
a.sum()

4.8595784

In [125]:
# Code here
a.min()

0.05093587

You can specify on which axis you want the aggregation function to be computed.

**Try To**
- find the minimum value within each column by specifying `axis=0`.

In [126]:
# Code here
a.min(axis=0)

array([0.12697628, 0.05093587, 0.26590556, 0.5510652 ])

The four values listed above correspond to the number of columns in your array. With a four-column array, you will get four values as your result.

**Read more about [array methods here](https://numpy.org/doc/stable/reference/arrays.ndarray.html#array-ndarray-methods).**

---
# Section 3: Creating matrices
You can pass Python lists of lists to create a `2-D array` (or `matrix`) to represent them in NumPy.


In [127]:
data = np.array([[1, 2], [3, 4],[5,6]])
data

array([[1, 2],
       [3, 4],
       [5, 6]])

![alt_text](S1-pics\arr.png)

Indexing and slicing operations are useful when you’re manipulating matrices:

**Try It**
- Select 1st element in 2nd column
- Select all elements starting from 2nd row till the end
- Select 1st and secomnd elemnts in the 1st row

![alt_text](S1-pics\ind.png)

In [128]:
# Select 1st element in 2nd column
first_element_second_column = data[0, 1]
first_element_second_column

2

In [129]:
# Select all elements starting from 2nd row till the end
elements_from_2nd_row = data[1:, :]
elements_from_2nd_row

array([[3, 4],
       [5, 6]])

In [130]:
# Select 1st and secomnd elemnts in the 1st row
first_two_elements_first_row = data[0, :2]
first_two_elements_first_row

array([1, 2])

You can aggregate matrices the same way you aggregated vectors:

**Try to**
- Get `max`, `min`, and `sum`
![alt_text](S1-pics\agg.png)

In [131]:
# max

max_value = np.max(data)

max_value

6

In [132]:
# min
min_value = np.min(data)
min_value

1

In [133]:
# sum
sum_value = np.sum(data)
sum_value

21

You can aggregate all the values in a matrix and you can aggregate them across columns or rows using the `axis` parameter:
    
**Try to**
- get `max` across both axes

![alt_text](S1-pics\axi-max.png)

In [134]:
# max across rows
max_values_across_rows = np.amax(data, axis=1)
max_values_across_rows

array([2, 4, 6])

In [135]:
# max across columns
max_values_across_rows = np.amax(data, axis=0)
max_values_across_rows

array([5, 6])

### Adding matricies
Once you’ve created your matrices, you can add and multiply them using arithmetic operators if you have two matrices that are the same size.

![alt_text](S1-pics\addmat.png)

In [136]:
data = np.array([[1, 2], [3, 4]])
ones = np.array([[1, 1], [1, 1]])
data + ones

array([[2, 3],
       [4, 5]])

You can do these arithmetic operations on matrices of different sizes, but only if one matrix has only one column or one row. In this case, NumPy will use its broadcast rules for the operation.

![alt_text](S1-pics\addrow.png)

In [137]:
data = np.array([[1, 2], [3, 4], [5, 6]])
ones_row = np.array([[1, 1]])
data + ones_row

array([[2, 3],
       [4, 5],
       [6, 7]])

Be aware that when NumPy prints N-dimensional arrays, the last axis is looped over the fastest while the first axis is the slowest. For instance:

In [138]:
np.ones((4, 3, 2))

array([[[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.]]])

There are often instances where we want NumPy to initialize the values of an array.

NumPy offers functions like `ones()` and `zeros()`, and the `random.Generator` class for random number generation for that.

All you need to do is pass in the number of elements you want it to generate:

![alt_text](S1-pics\10r.png)

In [139]:
np.ones(3)

array([1., 1., 1.])

In [140]:
np.zeros(3)

array([0., 0., 0.])

In [141]:
# the simplest way to generate random numbers
rng = np.random.default_rng(0)
rng.random(3)

array([0.63696169, 0.26978671, 0.04097352])

---
You can also use `ones()`, `zeros()`, and `random()` to create a `2D array` if you give them a `tuple` describing the dimensions of the matrix:
![alt_text](S1-pics\tuple.png)

In [142]:
np.ones((3, 2))

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [143]:
np.zeros((3, 2))

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [144]:
rng.random((3, 2))

array([[0.01652764, 0.81327024],
       [0.91275558, 0.60663578],
       [0.72949656, 0.54362499]])

---
# Section 4: Generating random numbers
The use of random number generation is an important part of the configuration and evaluation of many numerical and machine learning algorithms.

Whether you need to **randomly initialize weights** in an artificial neural network, **split data into random sets**, or **randomly shuffle your dataset**, being able to generate random numbers (actually, repeatable pseudo-random numbers) is essential.

With `Generator.integers`, you can generate random integers from low (remember that this is inclusive with NumPy) to high (exclusive). You can set `endpoint=True` to make the high number inclusive.

You can generate a `2 x 4 array` of random integers between `0 and 4` with:

In [145]:
rng.integers(5, size=(2, 4))

array([[2, 4, 1, 4],
       [3, 0, 1, 4]])

**[Read more about random number generation here.](https://numpy.org/doc/stable/reference/random/index.html#numpyrandom)**

---
# Section 5: How to get unique items and counts

*This section covers `np.unique()`*

---
You can find the unique elements in an array easily with `np.unique`.

**For example**, if you start with this array:

In [146]:
a = np.array([11, 11, 12, 13, 14, 15, 16, 17, 12, 13, 11, 14, 18, 19, 20])

you can use `np.unique` to print the unique values in your array:

In [147]:
unique_values = np.unique(a)
unique_values

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

---
To get the `indices` of unique values in a NumPy array (an array of ***first index positions*** of unique values in the array), just set the `return_index` argument as `True` in `np.unique()` as well as your array.

**Try it**

In [148]:
# Code here
unique_values, indices_list =np.unique(a, return_index=True)
indices_list

array([ 0,  2,  3,  4,  5,  6,  7, 12, 13, 14])

---
You can pass the `return_counts` argument in `np.unique()` along with your array to get the frequency `count` of unique values in a NumPy array.

**Try It**

In [149]:
# Code here
unique_values, occurrence_count =np.unique(a, return_counts=True)
occurrence_count

array([3, 2, 2, 2, 1, 1, 1, 1, 1, 1])

---
This also works with 2D arrays! If you start with this array:

In [150]:
a_2d = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [1, 2, 3, 4]])
a_2d

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [ 1,  2,  3,  4]])

In [151]:
unique_values = np.unique(a_2d)
unique_values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

> If the axis argument isn’t passed, your 2D array will be flattened.
---

If you want to get the `unique rows or columns`, make sure to pass the `axis` argument. To find the unique rows, specify `axis=0` and for columns, specify `axis=1`.

**Like this**

In [152]:
unique_rows = np.unique(a_2d, axis=0)
unique_rows

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

---
**Try this**
- To get the unique rows, index position, and occurrence count, you can use:

In [153]:
# code here
unique_rows, indices, occurrence_count = np.unique(data, axis=0, return_index=True, return_counts=True)
print(unique_rows)
print(indices)
print(occurrence_count)

[[1 2]
 [3 4]
 [5 6]]
[0 1 2]
[1 1 1]


**[Learn more about `numpy.unique` here.](https://numpy.org/doc/stable/reference/generated/numpy.unique.html#numpy.unique)**

---
# Section 6: Transposing and reshaping a matrix
*This section covers `arr.reshape()`, `arr.transpose()`, `arr.T`*

It’s common to need to transpose your matrices. NumPy arrays have the property `T` that allows you to transpose a matrix.

![alt_text](S1-pics\t1.png)

You may also need to `switch the dimensions` of a matrix.

>This can happen when, for example, you have a model that expects a certain input shape that is different from your dataset.

This is where the reshape method can be useful. You simply need to pass in the new dimensions that you want for the matrix.

In [154]:
data = np.arange(1,7)
data

array([1, 2, 3, 4, 5, 6])

In [155]:
data.reshape(2, 3)

array([[1, 2, 3],
       [4, 5, 6]])

In [156]:
data.reshape(3, 2)

array([[1, 2],
       [3, 4],
       [5, 6]])


![alt_text](S1-pics\t2.png)

You can also use `.transpose()` to reverse or change the axes of an array according to the values you specify.

If you start with this array:

In [157]:
arr = np.arange(6).reshape((2, 3))
arr

array([[0, 1, 2],
       [3, 4, 5]])

You can transpose your array with `arr.transpose()`.

In [158]:
arr.transpose()

array([[0, 3],
       [1, 4],
       [2, 5]])

You can also use `arr.T`:

In [159]:
arr.T

array([[0, 3],
       [1, 4],
       [2, 5]])

**To learn more about transposing and reshaping arrays, see [transpose](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html#numpy.transpose) and [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape).**

---
# Section 7: How to reverse an array
*This section covers `np.flip()`*

---
NumPy’s `np.flip()` function allows you to ***flip, or reverse***, the contents of an array along an axis.

- When using `np.flip()`, specify the array you would like to reverse and the `axis`.
- If you don’t specify the axis, NumPy will reverse the contents along all of the axes of your input array.

### Reversing a 1D array

If you begin with a 1D array like this one:

In [160]:
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8])
arr

array([1, 2, 3, 4, 5, 6, 7, 8])

In [161]:
# You can reverse it with:
reversed_arr = np.flip(arr)
reversed_arr

array([8, 7, 6, 5, 4, 3, 2, 1])

---
### Reversing a 2D array

A 2D array works much the same way.

If you start with this array:

In [162]:
arr_2d = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
arr_2d

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [163]:
# You can reverse the content in all of the rows and all of the columns with:
reversed_arr = np.flip(arr_2d)
reversed_arr

array([[12, 11, 10,  9],
       [ 8,  7,  6,  5],
       [ 4,  3,  2,  1]])

In [164]:
# You can easily reverse only the rows with:
reversed_arr_rows = np.flip(arr_2d, axis=0)
reversed_arr_rows

array([[ 9, 10, 11, 12],
       [ 5,  6,  7,  8],
       [ 1,  2,  3,  4]])

In [165]:
# Or reverse only the columns with:
reversed_arr_columns = np.flip(arr_2d, axis=1)
reversed_arr_columns

array([[ 4,  3,  2,  1],
       [ 8,  7,  6,  5],
       [12, 11, 10,  9]])

>You can also reverse the contents of only one column or row. For example, you can reverse the contents of the row at index position 1 (the second row):


In [166]:
arr_2d[1] = np.flip(arr_2d[1])
arr_2d

array([[ 1,  2,  3,  4],
       [ 8,  7,  6,  5],
       [ 9, 10, 11, 12]])

>You can also reverse the column at index position 1 (the second column):

In [167]:
arr_2d[:,1] = np.flip(arr_2d[:,1])
arr_2d

array([[ 1, 10,  3,  4],
       [ 8,  7,  6,  5],
       [ 9,  2, 11, 12]])

**Read more about reversing arrays at [flip](https://numpy.org/doc/stable/reference/generated/numpy.flip.html#numpy.flip).**

---
# Section 9: Reshaping and flattening multidimensional arrays
*This section covers `.flatten()`, `ravel()`*

There are two popular ways to flatten an array: `.flatten()` and `.ravel()`.

>The primary difference between the two is that ***the new array created using ravel() is actually a reference to the parent array (i.e., a “view”)***.

This means that any changes to the new array will affect the parent array as well. Since ravel does not create a copy, it’s memory efficient.

If you start with this array:

In [168]:
x = np.array([[1 , 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
x

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

**Try to**
- Use `flatten` to flatten your array into a 1D array.

In [169]:
# Code here
x.flatten()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

> **Note that:** When you use `flatten`, changes to your new array won’t change the parent array.

For example:

In [170]:
a1 = x.flatten()
a1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [171]:
a1[0] = 99

In [172]:
print(x)  # Original array

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [173]:
print(a1)  # New array

[99  2  3  4  5  6  7  8  9 10 11 12]


>**BUT**: But when you use `ravel`, the changes you make to the new array will affect the parent array.

For example:

In [174]:
a2 = x.ravel()
a2

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [175]:
a2[0] = 98

In [176]:
print(x)  # Original array

[[98  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [177]:
print(a2)  # New array

[98  2  3  4  5  6  7  8  9 10 11 12]


**Read more about `flatten` at [ndarray.flatten](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html#numpy.ndarray.flatten) and `ravel` at [ravel](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html#numpy.ravel).**

---
# Section 10: How to access the docstring for more information
*This section covers `help()`, `?`, `??`*

---

When it comes to the data science ecosystem, Python and NumPy are built with the user in mind. One of the best examples of this is the built-in access to documentation.

Every object contains the reference to a string, which is known as the `docstring`.

In most cases, this `docstring` contains a quick and concise `summary of the object` and `how to use it`.

Python has a built-in `help()` function that can help you access this information.

This means that nearly any time you need more information, you can use `help()` to quickly find the information that you need.

**For example:**

In [178]:
help(max)

Help on built-in function max in module builtins:

max(...)
    max(iterable, *[, default=obj, key=func]) -> value
    max(arg1, arg2, *args, *[, key=func]) -> value
    
    With a single iterable argument, return its biggest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the largest argument.



Because access to additional information is so useful, **IPython** uses the `?` character as a shorthand for accessing this documentation along with other relevant information.

**IPython** is a command shell for interactive computing in multiple languages. **[You can find more information about IPython here](https://ipython.org/)**.

**For example:**

In [179]:
max?

Docstring:
max(iterable, *[, default=obj, key=func]) -> value
max(arg1, arg2, *args, *[, key=func]) -> value

With a single iterable argument, return its biggest item. The
default keyword-only argument specifies an object to return if
the provided iterable is empty.
With two or more arguments, return the largest argument.
Type:      builtin_function_or_method

![alt_text](S1-pics\help.png)

You can even use this notation for object methods and objects themselves.

Let’s say you create this array:

In [180]:
import numpy as np
a = np.array([1, 2, 3, 4, 5, 6])
a?

Type:        ndarray
String form: [1 2 3 4 5 6]
Length:      6
File:        ~/.local/lib/python3.8/site-packages/numpy/__init__.py
Docstring:  
ndarray(shape, dtype=float, buffer=None, offset=0,
        strides=None, order=None)

An array object represents a multidimensional, homogeneous array
of fixed-size items.  An associated data-type object describes the
format of each element in the array (its byte-order, how many bytes it
occupies in memory, whether it is an integer, a floating point number,
or something else, etc.)

Arrays should be constructed using `array`, `zeros` or `empty` (refer
to the See Also section below).  The parameters given here refer to
a low-level method (`ndarray(...)`) for instantiating an array.

For more information, refer to the `numpy` module and examine the
methods and attributes of an array.

Parameters
----------
(for the __new__ method; see Notes below)

shape : tuple of ints
    Shape of created array.
dtype : data-type, optional
    Any object that c

Then you can obtain a lot of useful information (first details about a itself, followed by the docstring of ndarray of which a is an instance):

![alt_text](S1-pics\h2.png)

This also works for **functions** and other objects that you create.

>Just remember to include a docstring with your function using a string literal (`""" """` or `''' '''` around your documentation).

For example, if you create this function:

In [181]:
def double(a):
   '''Return a * 2'''
   return a * 2

In [182]:
double?

Signature: double(a)
Docstring: Return a * 2
File:      /tmp/ipykernel_20331/3919353596.py
Type:      function

![alt_text](S1-pics\h3.png)

You can reach another level of information by reading the source code of the object you’re interested in. Using a double question mark (`??`) allows you to access the source code.

For example:

In [183]:
double??

Signature: double(a)
Source:   
def double(a):
   '''Return a * 2'''
   return a * 2
File:      /tmp/ipykernel_20331/3919353596.py
Type:      function

![alt_text](S1-pics\h4.png)

If the object in question is compiled in a language other than Python, using `??` will return the same information as `?`.

You’ll find this with a lot of built-in objects and types, for example: `len?` and `len??` returns the same info.

***Why they have the same output?***
>because they were compiled in a programming language other than Python.

---
# Section 11: How to save and load NumPy objects
*This section covers `np.save`, `np.savez`, `np.savetxt`, `np.load`, `np.loadtxt`*

---

You will, at some point, want to save your arrays to disk and load them back without having to re-run the code. Fortunately, there are several ways to save and load objects with NumPy.

- The `ndarray objects` can be saved to and loaded from the disk files with `loadtxt` and `savetxt` functions that handle normal **text files**,
- `load` and `save` functions that handle NumPy binary files with a `.npy` **file extension**,
- and a `savez` function that handles NumPy files with a `.npz` **file extension**.

>The `.npy` and `.npz` files store `data`, `shape`, `dtype`, and ***other information required to reconstruct the ndarray in a way that allows the array to be correctly retrieved***, even when the file is on another machine with different architecture.

- If you want to store a *single ndarray object*, store it as a `.npy file` using `np.save`.
- If you want to store *more than one ndarray object in a single file*, save it as a `.npz file` using `np.savez`.
- You can also save *several arrays into a single file* in **compressed npz format** with `savez_compressed`.

It’s easy to save and load and array with `np.save()`. Just make sure to specify the array you want to save and a file name.

**For example**, if you create this array:


In [184]:
a = np.array([1, 2, 3, 4, 5, 6])

In [185]:
# You can save it as “filename.npy” with:
np.save('numpy ', a)

In [186]:
# You can use np.load() to reconstruct your array.
b = np.load('numby.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'numby.npy'

In [187]:
# If you want to check your array, you can run::
print(b)

[0 1 2]


You can save a NumPy array as a **plain text file** like a *.csv* or *.txt* file with `np.savetxt`.

**For example**, if you create this array:

In [188]:
csv_arr = np.array([1, 2, 3, 4, 5, 6, 7, 8])

In [189]:
# You can easily save it as a .csv file with the name “new_file.csv” like this:
np.savetxt('new_file.csv', csv_arr)

In [190]:
# You can quickly and easily load your saved text file using loadtxt():
np.loadtxt('new_file.csv')

array([1., 2., 3., 4., 5., 6., 7., 8.])

> The `savetxt()` and `loadtxt()` functions accept additional optional parameters such as *header*, *footer*, and *delimiter*.
    
While text files can be easier for sharing, `.npy` and `.npz` files are smaller and faster to read.

>If you need more sophisticated handling of your text file (for example, if you need to work with lines that contain missing values), you will want to use the `genfromtxt` function.

Learn more about [input and output routines here](https://numpy.org/doc/stable/reference/routines.io.html#routines-io).



# Thank you for your time and efforts!
### *By:Heba-T-ALLAH Raslan*